# Task 3

# Importing packages

In [1]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
print(tf.__version__)

2.8.0


# Resizing the images

In [5]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder
IMG_SHAPE = 224
for filename in os.listdir('food'):
    img = Image.open('food/' + filename)
    img = img.resize((IMG_SHAPE, IMG_SHAPE))
    img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# setting up the triplets for deep ranking

In [ ]:
line_id = 0
with open('train_triplets.txt', 'r') as f:
    # go through the lines
    for line in f:
        line = line.replace("\n", "")

        shutil.copy("food_224/" + line.split(' ')[0] + ".jpg","triplets/class1/" + str(line_id) + ".jpg")
        line_id += 1
        shutil.copy("food_224/" + line.split(' ')[1] + ".jpg","triplets/class1/" + str(line_id) + ".jpg")
        line_id += 1
        shutil.copy("food_224/" + line.split(' ')[2] + ".jpg","triplets/class1/" + str(line_id) + ".jpg")
        line_id += 1

        if line_id % 1000 == 0:
            print(line_id)
            break

# everything below is no longer needed (it was used to extract features with mobilenet, to get that 0.61 score)

# Extracting the features

In [6]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL, input_shape=(IMG_SHAPE, IMG_SHAPE,3))
feature_extractor.trainable = False

model = tf.keras.Sequential([
  feature_extractor,
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
img = np.array(Image.open('food_224/00001.jpg'))[None]
print(img.shape)
output = model.predict(img)
print(output.shape)

(1, 224, 224, 3)
(1, 1280)


In [63]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_224'):
    line_id += 1
    img = Image.open('food_224/' + filename)
    feature = model.predict(np.array(img)[None])[0]
    features[filename.replace(".jpg","")] = feature
    if line_id % 100 == 0:
        print(line_id)
    # break

# save the features dictionary to a file using pickle
with open('features.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [10]:
output_shape = 1280
features = {}
# load the features dictionary from the file
with open('features.pickle', 'rb') as handle:
    features = pickle.load(handle)
print(features['02461'])

[0.01322182 1.4992157  0.03471313 ... 0.1179836  0.00344099 0.57712704]


In [14]:
# each line of train_triplets.txt contains 3 images, separated by a space
# the first 2 images are the anchor and positive examples, and the third is the negative example
# extract the features of the 3 images and save them in a file
cols = columns=np.array(['x' + str(i) for i in np.arange(output_shape*2)+1])
df_features = pd.DataFrame(columns=cols)
df_labels = pd.DataFrame(columns=['label'])

line_id = 0
with open('train_triplets.txt', 'r') as f:
    # go through the lines
    for line in f:
        line_id += 1
        line = line.replace("\n", "")
        anchor = features[line.split(' ')[0]]
        positive = features[line.split(' ')[1]]
        negative = features[line.split(' ')[2]]
        if random.random() < 0.5:
            row = np.concatenate((positive-anchor, negative-anchor))
            df_labels = df_labels.append(pd.DataFrame(np.array([[1]]), columns=['label']))
        else:
            row = np.concatenate((negative-anchor, positive-anchor))
            df_labels = df_labels.append(pd.DataFrame(np.array([[0]]), columns=['label']))
        # add the new row to the dataframe
        df_features = df_features.append(pd.DataFrame(row.reshape(1, -1), columns=cols))
        if line_id % 5000 == 0:
            print(line_id)
            # break
    
print(df_features.shape)
print(df_labels.shape)
df_features.to_csv('train_features.csv',index=False, float_format='%.3f')
df_labels.to_csv('train_labels.csv',index=False)


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
(59515, 2560)
(59515, 1)
